# TPOT on Dask on CDSW Workers

In [1]:
!pip3 install dask[complete]==2021.1.1 dask-ml==1.8.0 tpot==0.11.7

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import os
import time

import cdsw
from dask.distributed import Client
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier

/home/cdsw/.local/lib/python3.6/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
os.makedirs("_scheduler_", exist_ok=True)
os.makedirs("_worker_", exist_ok=True)

## Start Dask scheduler

In [3]:
dask_scheduler = cdsw.launch_workers(
  n=1,
  cpu=1,
  memory=2,
  kernel="python3",
  code=f"!dask-scheduler --host 0.0.0.0 --dashboard-address 8080 --scheduler-file _scheduler_/dask.log"
)

Wait for the scheduler to start.

In [4]:
time.sleep(10)

Find the IP address of the scheduler.

In [6]:
scheduler_workers = cdsw.list_workers()
scheduler_id = dask_scheduler[0]['id']
scheduler_ip = [worker['ip_address'] for worker in scheduler_workers
                if worker['id'] == scheduler_id][0]

scheduler_url = f"tcp://{scheduler_ip}:8786"

scheduler_url

'tcp://100.66.176.11:8786'

## Start Dask workers

In [7]:
dask_workers = cdsw.launch_workers(
  n=10,
  cpu=1,
  memory=2,
  kernel="python3",
  code=f"!dask-worker {scheduler_url} --local-directory _worker_"
)

# Wait for the workers to start.
time.sleep(10)

## Connect Dask client

In [8]:
client = Client(scheduler_url)

In [9]:
client

Client Scheduler: tcp://100.66.176.11:8786 Dashboard: http://100.66.176.11:8080/status,Cluster Workers: 10 Cores: 80 Memory: 20.00 GB


## Load data

In [10]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, train_size=0.75, test_size=0.25)

## Define estimator (using Dask!)

In [12]:
estimator = TPOTClassifier(generations=5, population_size=20, use_dask=True)

## Fit estimator (using Dask workers!)

In [13]:
estimator.fit(X_train, y_train)

TPOTClassifier(generations=5, population_size=20, use_dask=True)

In [14]:
estimator.predict(X_train)

array([9, 2, 4, ..., 4, 0, 1])

## Close workers

Stop workers. Stop only those that we started, not all the workers on the cluster, that others may be using.

In [15]:
cdsw.stop_workers(*[worker['id'] for worker in dask_workers])

[<Response [204]>,
 <Response [204]>,
 <Response [204]>,
 <Response [204]>,
 <Response [204]>,
 <Response [204]>,
 <Response [204]>,
 <Response [204]>,
 <Response [204]>,
 <Response [204]>]

Stop scheduler.

In [16]:
cdsw.stop_workers(*[worker['id'] for worker in dask_scheduler])

[<Response [204]>]